<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import datetime

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
data = {
    'mktId': 'KSQ',
    'trdDd': today,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901',
}
gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_key = requests.post(gen_url, data=data)

In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
r = requests.post(down_url, data={'code':gen_key.text})
r.encoding = 'EUC-KR'

In [5]:
from io import StringIO

df = pd.read_csv(StringIO(r.text))

In [6]:
df

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,3950,-35,-0.88,182772476350
1,054620,APS홀딩스,KOSDAQ,금융,15950,250,1.59,325287824950
2,265520,AP시스템,KOSDAQ,반도체,24950,-1100,-4.22,361281663650
3,211270,AP위성,KOSDAQ,통신장비,16850,-600,-3.44,254136822400
4,013720,CBI,KOSDAQ,운송장비·부품,2750,-580,-17.42,139418881250
...,...,...,...,...,...,...,...,...
1511,024060,흥구석유,KOSDAQ,유통,7960,-90,-1.12,119400000000
1512,010240,흥국,KOSDAQ,기계·장비,9100,-230,-2.47,112136533600
1513,189980,흥국에프엔비,KOSDAQ,음식료·담배,3230,-110,-3.29,123990420210
1514,037440,희림,KOSDAQ,기타서비스,6650,-210,-3.06,92584458750


In [7]:
'''
%%timeit
df.loc[df['시가총액'].between(30000000000,70000000000), '종목명']

%%timeit
df[df['시가총액'].between(30000000000,70000000000)]['종목명']
'''

"\n%%timeit\ndf.loc[df['시가총액'].between(30000000000,70000000000), '종목명']\n\n%%timeit\ndf[df['시가총액'].between(30000000000,70000000000)]['종목명']\n"

In [8]:
boundary = df.loc[df['시가총액'].between(30000000000,70000000000)]
boundary

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
10,056000,COWON,KOSDAQ,디지털컨텐츠,2860,0,0.00,56399843500
12,083660,CSA 코스믹,KOSDAQ,화학,1480,-5,-0.34,56142488720
20,050120,ES큐브,KOSDAQ,유통,4535,-65,-1.41,61513130010
21,130500,GH신소재,KOSDAQ,섬유·의류,6270,-340,-5.14,68970000000
22,900290,GRT,KOSDAQ,금융,951,0,0.00,64073625000
...,...,...,...,...,...,...,...,...
1467,002680,한탑,KOSDAQ,음식료·담배,1325,-40,-2.93,37457734100
1475,220180,핸디소프트,KOSDAQ,소프트웨어,3605,-140,-3.74,69965840000
1476,900270,헝셩그룹,KOSDAQ,금융,489,-11,-2.20,39120000000
1489,011080,형지I&C,KOSDAQ,섬유·의류,1370,-40,-2.84,53369265160


In [9]:
!pip install OpenDartReader

In [10]:
import OpenDartReader

api_key = '4b0512c473e52e1cf59f8ce8380b2b30904f381a'
dart = OpenDartReader(api_key) 

In [11]:
'''
search_start = '2021-08-01'
search_end = '2021-10-01'
'''

search_date = datetime.date.today().strftime('%Y-%m-%d')

"\nsearch_start = '2021-08-01'\nsearch_end = '2021-10-01'\n"

In [12]:
dart_df = dart.list(start=search_date, end=search_date, kind='A')
found = list(set(dart_df['stock_code']).intersection(set(boundary['종목코드'])))
out_df = dart_df[dart_df['stock_code'].isin(found)]

{"status":"013","message":"조회된 데이타가 없습니다."}


KeyError: ignored

In [ ]:
def get_recent_finstate(corp_code='', year=2021):
  quarters = ['11011', '11014', '11012', '11013']
  for q in quarters:
    # '11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서
    df = dart.finstate(corp_code, year, reprt_code=q)
    if type(df) != type(None):
      return df[df['fs_nm'] == '연결재무제표']

In [ ]:
def get_incomestatement(df):
  is_df = df.loc[df['sj_nm'] == '손익계산서']
  try:
    t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount', 'thstrm_add_amount', 'frmtrm_add_amount']]
    t.columns = ['published_date', 'date', 'name', 'current03', 'previous03', 'current12', 'previous12']
  except:
    t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
    t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_is_score(is_df):
  is_score = {}
  for n in is_df['name']:
    current = str_int(is_df.loc[is_df['name'] == n, 'current03'].values[0])
    previous = str_int(is_df.loc[is_df['name'] == n, 'previous03'].values[0])
    is_score[n] = get_growth_rate(previous, current)
  return is_score

In [ ]:
def get_balancesheet(df):
  bs_df = df.loc[df['sj_nm'] == '재무상태표']
  t = bs_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
  t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_bs_score(bs_df):
  bs_score = {}
  for n in bs_df['name']:
    current = str_int(bs_df.loc[bs_df['name'] == n, 'current03'].values[0])
    previous = str_int(bs_df.loc[bs_df['name'] == n, 'previous03'].values[0])
    bs_score[n] = get_growth_rate(previous, current)
  return bs_score

In [ ]:
def get_growth_rate(previous, current):
  return (current-previous)/abs(previous)

def str_int(s):
  return int(s.replace(',',''))

In [ ]:
df = get_recent_finstate('290270')
is_df = get_incomestatement(df)
is_score = get_is_score(is_df)

bs_df = get_balancesheet(df)
bs_score = get_bs_score(bs_df)

In [ ]:
for code in out_df['corp_code']:
  try:
    df = get_recent_finstate(code)
    is_df = get_incomestatement(df)
    is_score = get_is_score(is_df)

    bs_df = get_balancesheet(df)
    bs_score = get_bs_score(bs_df)

    date = out_df.loc[out_df['corp_code'] == code, 'rcept_dt'].values[0]
    name = out_df.loc[out_df['corp_code'] == code, 'corp_name'].values[0]
    print(date)
    print(name)
    print(is_score)
    print(bs_score)
    print()

    t = is_score
    t.update(bs_score)
    score_dic[name] = t
  except:
    print('ERROR', name)
    print()

In [ ]:
score_df = pd.DataFrame(score_dic).T
score_df

In [ ]:
score_df.sort_values(by=['영업이익', '매출액'], ascending=False).head(10)